<a href="https://colab.research.google.com/github/trick491/Data-Science-Final-Project/blob/main/data_wrangling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Pull Together Data**

In [1]:
import pandas as pd
import requests
from io import StringIO

edgar_api = 'https://api.github.com/repos/trick491/Data-Science-Final-Project/contents/data/edgar'
yahoo_api = 'https://api.github.com/repos/trick491/Data-Science-Final-Project/contents/data/yahoo'

edgar_response = requests.get(edgar_api)
yahoo_response = requests.get(yahoo_api)

edgar_df = pd.DataFrame()
yahoo_df = pd.DataFrame()

if edgar_response.status_code == 200:
    edgar_files = edgar_response.json()
    for file in edgar_files:
        if file['name'].endswith('.csv'):
            file_url = file['download_url']
            df_temp = pd.read_csv(file_url)
            edgar_df = pd.concat([edgar_df, df_temp], ignore_index=True)
            print(f"Loaded EDGAR file: {file['name']}")

if yahoo_response.status_code == 200:
    yahoo_files = yahoo_response.json()
    for file in yahoo_files:
        if file['name'].endswith('.csv'):
            file_url = file['download_url']
            df_temp = pd.read_csv(file_url)
            yahoo_df = pd.concat([yahoo_df, df_temp], ignore_index=True)
            print(f"Loaded Yahoo file: {file['name']}")

print(f"\nEDGAR DataFrame shape: {edgar_df.shape}")
print(f"Yahoo DataFrame shape: {yahoo_df.shape}")

Loaded EDGAR file: edgar_1.csv
Loaded EDGAR file: edgar_10.csv
Loaded EDGAR file: edgar_11.csv
Loaded EDGAR file: edgar_12.csv
Loaded EDGAR file: edgar_13.csv
Loaded EDGAR file: edgar_14.csv
Loaded EDGAR file: edgar_15.csv
Loaded EDGAR file: edgar_16.csv
Loaded EDGAR file: edgar_17.csv
Loaded EDGAR file: edgar_18.csv
Loaded EDGAR file: edgar_19.csv
Loaded EDGAR file: edgar_2.csv
Loaded EDGAR file: edgar_20.csv
Loaded EDGAR file: edgar_21.csv
Loaded EDGAR file: edgar_22.csv
Loaded EDGAR file: edgar_23.csv
Loaded EDGAR file: edgar_24.csv
Loaded EDGAR file: edgar_25.csv
Loaded EDGAR file: edgar_26.csv
Loaded EDGAR file: edgar_27.csv
Loaded EDGAR file: edgar_28.csv
Loaded EDGAR file: edgar_29.csv
Loaded EDGAR file: edgar_3.csv
Loaded EDGAR file: edgar_30.csv
Loaded EDGAR file: edgar_31.csv
Loaded EDGAR file: edgar_32.csv
Loaded EDGAR file: edgar_33.csv
Loaded EDGAR file: edgar_34.csv
Loaded EDGAR file: edgar_35.csv
Loaded EDGAR file: edgar_36.csv
Loaded EDGAR file: edgar_37.csv
Loaded EDGA

Now we will create two rows with price before and after quarterlies

In [ ]:
from multiprocessing import Pool, cpu_count
import numpy as np
from tqdm import tqdm

print(edgar_df.head())
print(yahoo_df.head())
full_data = edgar_df.copy()

# Create a dictionary for faster lookups
print("Creating lookup dictionary...")
yahoo_dict = {}
for ticker in tqdm(yahoo_df['Ticker'].unique(), desc="Processing tickers"):
    ticker_data = yahoo_df[yahoo_df['Ticker'] == ticker].sort_values('Date')
    yahoo_dict[ticker] = ticker_data

def get_price_before(args):
    idx, ticker, filed_date = args

    if ticker not in yahoo_dict:
        return None

    ticker_data = yahoo_dict[ticker]
    matching = ticker_data[ticker_data['Date'] <= filed_date]
    return matching.iloc[-1]['Close'] if len(matching) > 0 else None

def get_price_after(args):
    idx, ticker, filed_date = args

    if ticker not in yahoo_dict:
        return None

    ticker_data = yahoo_dict[ticker]
    matching = ticker_data[ticker_data['Date'] > filed_date]
    return matching.iloc[0]['Close'] if len(matching) > 0 else None

# Prepare arguments for parallel processing
print("Preparing data...")
args_list = [(idx, row.Ticker, row.Filed) for idx, row in enumerate(full_data.itertuples())]

# Calculate Price before in parallel
num_cores = cpu_count()
print(f"\nUsing {num_cores} cores")
print("Calculating 'Price before' in parallel...")

with Pool(num_cores) as pool:
    results = list(tqdm(
        pool.imap(get_price_before, args_list, chunksize=1000),
        total=len(args_list),
        desc="Price before"
    ))
    full_data['Price before'] = results

print("✓ 'Price before' complete!\n")

# Calculate Price after in parallel
print("Calculating 'Price after' in parallel...")

with Pool(num_cores) as pool:
    results = list(tqdm(
        pool.imap(get_price_after, args_list, chunksize=1000),
        total=len(args_list),
        desc="Price after"
    ))
    full_data['Price after'] = results

print("✓ 'Price after' complete!\n")

print('****************** AFTER TRANSFORMATION ********************')

print(full_data.head())


  Ticker      CIK                      Label                   Title  \
0   GOOG  1652044  Accounts Payable, Current  AccountsPayableCurrent   
1   GOOG  1652044  Accounts Payable, Current  AccountsPayableCurrent   
2   GOOG  1652044  Accounts Payable, Current  AccountsPayableCurrent   
3   GOOG  1652044  Accounts Payable, Current  AccountsPayableCurrent   
4   GOOG  1652044  Accounts Payable, Current  AccountsPayableCurrent   

                                         Description Unit         Value  \
0  Carrying value as of the balance sheet date of...  USD  6.037000e+09   
1  Carrying value as of the balance sheet date of...  USD  6.037000e+09   
2  Carrying value as of the balance sheet date of...  USD  6.037000e+09   
3  Carrying value as of the balance sheet date of...  USD  6.037000e+09   
4  Carrying value as of the balance sheet date of...  USD  6.037000e+09   

         Date       Filed  Fiscal Year Fiscal Period  Form  
0  2021-12-31  2022-02-02       2021.0            FY  1

Processing tickers: 100%|██████████| 501/501 [03:17<00:00,  2.54it/s]


Preparing data...

Using 2 cores
Calculating 'Price before' in parallel...


Price before:  73%|███████▎  | 1945001/2667269 [47:10<11:22, 1058.86it/s]

***